In [11]:
import pandas as pd
import numpy as np
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from tqdm import tqdm  # Visualize loop progress
# from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed
import openpyxl
import os
import requests
import yfinance as yf

import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

In [12]:
# tradingview lightweight-charts
from lightweight_charts import Chart
import asyncio
import nest_asyncio
# Apply nest_asyncio AT START to allow nested event loops (required for Jupyter)
nest_asyncio.apply()
#
from dotenv import load_dotenv

In [13]:
# load env vars
load_dotenv()

True

In [14]:
# #####---------------------#####s
# screen_date = '2025-03-07'
# screen_date = '2025-03-13'
# screen_date = '2025-03-19'
# screen_date = '2025-03-20'
screen_date = '2025-03-21'
# #####---------------------#####
# read pre market data from TradingView data via csv
trading_view_df = pd.read_csv(f"/Users/sudz4/Desktop/MATT-MATTHEW/matt-matthew/tv_screen_gap-up_{screen_date}.csv")

In [15]:
# print(trading_view_df.columns)

In [16]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()
# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume',
    'Exponential Moving Average (3) 1 day', 'Exponential Moving Average (7) 1 day'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    # stock filtering conditions >=< based on criteria
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes',
    'Exponential Moving Average (3) 1 day',
    'Exponential Moving Average (7) 1 day'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stocks found.")
display(smash_df.head(20))
# display(smash_df)
# # ALL SMASH
# display(smash_df)

27 stocks found.


,Symbol,Description,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Industry,Index,Sector,Exchange,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time,Analyst Rating,Technical Rating 5 minutes,Exponential Moving Average (3) 1 day,Exponential Moving Average (7) 1 day
0,ACVA,ACV Auctions Inc.,Mid caps,6.091718,1.711157,2.465659e+09,14.61,14.8600,Internet retail,"NASDAQ Composite, Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, NASDAQ Industrials, Mini-Russell 2000",Retail trade,NASDAQ,2025-02-19,2025-05-07,1.608373e+08,3379516.3,2.608346e+06,1.866736e+06,0.974638,5.133490,2.640573,0.863608,Strong buy,Sell,14.438864,14.396629
1,SGHC,Super Group (SGHC) Limited,Mid caps,4.892086,0.000000,3.484948e+09,6.95,6.9500,Casinos/Gaming,"Russell 2000, Russell 3000, Mini-Russell 2000",Consumer services,NYSE,2025-02-25,2025-05-21,1.479111e+08,916528.6,9.301919e+05,7.834150e+05,1.314391,5.297589,1.834837,1.019478,Strong buy,Buy,6.844398,6.823597
2,MUR,Murphy Oil Corporation,Mid caps,3.867528,3.867528,3.952676e+09,27.10,28.1481,Oil & gas production,"Russell 2000, S&P MidCap 400, Russell 3000, Mini-Russell 2000",Energy minerals,NYSE,2025-01-30,2025-04-30,1.374239e+08,3018169.3,3.012502e+06,2.338137e+06,0.834109,7.028155,3.903672,0.760053,Buy,Sell,26.599687,26.125586
3,PAM,Pampa Energia S.A.,Mid caps,3.814884,3.814884,5.265302e+09,79.95,83.0000,Electric utilities,NaN,Utilities,NYSE,2025-03-05,2025-05-07,5.434415e+07,217540.5,2.361458e+05,2.829342e+05,1.009855,6.716158,4.989576,1.017379,Buy,Strong buy,79.038341,78.844168
4,TTMI,"TTM Technologies, Inc.",Mid caps,3.538257,3.228660,2.297655e+09,22.61,23.3400,Electronic components,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Computer, Mini-Russell 2000",Electronic technology,NASDAQ,2025-02-05,2025-04-30,9.860428e+07,627188.4,8.064386e+05,6.698905e+05,1.455261,3.666541,2.768853,0.590550,Strong buy,Neutral,22.241006,22.183335
5,BTSG,"BrightSpring Health Services, Inc.",Mid caps,3.053435,3.053435,3.209020e+09,18.34,18.9000,Medical/Nursing services,"NASDAQ Composite, Russell 2000, Russell 3000, Mini-Russell 2000",Health services,NASDAQ,2025-03-06,2025-05-01,6.817049e+07,1939540.7,1.549713e+06,1.453171e+06,0.632598,5.770084,5.468411,0.573120,Strong buy,Neutral,17.940283,17.702095
6,GEO,Geo Group Inc (The) REIT,Mid caps,2.828541,1.029464,3.954488e+09,28.17,28.4600,Miscellaneous commercial services,"Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000",Commercial services,NYSE,2025-02-27,2025-04-30,1.321527e+08,3207725.0,2.905833e+06,3.460521e+06,0.904084,2.867109,3.994334,0.925340,Strong buy,Neutral,27.217105,26.435103
7,CAVA,"CAVA Group, Inc.",Mid caps,2.469136,1.851852,9.363162e+09,81.00,82.5000,Restaurants,"Nasdaq US Mid Cap Growth, Russell 3000, Russell 1000",Consumer services,NYSE,2025-02-25,2025-06-03,9.884469e+07,3715683.9,3.651291e+06,3.087164e+06,1.196362,4.940457,3.058632,1.054363,Buy,Neutral,79.233205,79.773580
8,PFSI,"PennyMac Financial Services, Inc.",Mid caps,2.009848,2.009848,5.118199e+09,99.51,101.5100,Finance/Rental/Leasing,"Russell 2000, Russell 3000, PHLX Housing Sector, Mini-Russell 2000",Finance,NYSE,2025-01-30,2025-05-01,4.887130e+07,311308.5,3.148802e+05,3.063680e+05,1.001399,7.442195,5.733812,0.987872,Buy,Neutral,98.803103,98.344994
9,AHR,"American Healthcare REIT, Inc.",Mid caps,1.393497,2.554745,4.809425e+09,30.14,30.9100,Real estate investment trusts,"Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000",Finance,NYSE,2025-02-27,2025-05-19,1.561765e+08,1810937.9,1.855938e+06,1.900138e+06,1.439056,3.860744,3.886682,0.963265,Strong buy,Neutral,29.775997,29.482624


In [17]:
# group and count by sector to new df called sector_count_df
sector_count_df = smash_df.groupby('Sector').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
# print the date and time of the screen
print(f"Screening date: {screen_date}")
print(f"Total stocks returned: {len(smash_df)}")
display(sector_count_df)

Screening date: 2025-03-20
Total stocks returned: 27


,Sector,Count
5,Finance,5
2,Consumer services,3
13,Utilities,3
1,Consumer durables,2
4,Energy minerals,2
7,Miscellaneous,2
9,Process industries,2
12,Technology services,2
0,Commercial services,1
3,Electronic technology,1


In [18]:
smash_list = smash_df['Symbol'].tolist()
print(len(smash_list))
print(smash_list)

27
['ACVA', 'SGHC', 'MUR', 'PAM', 'TTMI', 'BTSG', 'GEO', 'CAVA', 'PFSI', 'AHR', 'MDU', 'ENVA', 'VERX', 'MOS', 'TCBI', 'RRX', 'CSQ', 'TMHC', 'VFS', 'CAKE', 'ELP', 'USLM', 'DNB', 'GPOR', 'NVG', 'TBBK', 'PTVE']


In [19]:
# Create a filter for EMA3 > EMA7
smash_ema_df = smash_df['Exponential Moving Average (3) 1 day'] > smash_df['Exponential Moving Average (7) 1 day']

# Apply the filter to smash_df
filtered_smash_df = smash_df[smash_ema_df]

# Print the number of stocks that meet the criteria
print(f"Number of stocks where EMA3 > EMA7: {len(filtered_smash_df)}")
print("\nFiltered stocks:")
print(filtered_smash_df['Symbol'].tolist())

Number of stocks where EMA3 > EMA7: 24

Filtered stocks:
['ACVA', 'SGHC', 'MUR', 'PAM', 'TTMI', 'BTSG', 'GEO', 'PFSI', 'AHR', 'MDU', 'ENVA', 'VERX', 'MOS', 'TCBI', 'RRX', 'CSQ', 'VFS', 'CAKE', 'ELP', 'USLM', 'DNB', 'GPOR', 'TBBK', 'PTVE']
